# About this Kernel
This is my first attempt to publish a kernel in Kaggle. I will try H2O AutoML  on this IEEE Fraud transaction dataset. Primary goal for me is to work on a dataset that has combination of features (numbers, categorical, datetime) and the features are not explanatory, so i could experiment with pre-processing & Feature engineering concepts


## Credits
1. Thanks to kernels submitted by tunguz. I have used some of the concepts in this Kernel
https://www.kaggle.com/tunguz/ieee-with-h2o-automl


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import os
import gc
import itertools

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML
h2o.init(max_mem_size='12G')

3.26.0.2
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_222"; OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1~deb9u1-b10); OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)
  Starting server from /opt/conda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpykvhvpp3
  JVM stdout: /tmp/tmpykvhvpp3/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpykvhvpp3/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,19 days
H2O cluster name:,H2O_from_python_unknownUser_757x8h
H2O cluster total nodes:,1
H2O cluster free memory:,10.67 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [3]:
train_transaction=h2o.import_file("../input/train_transaction.csv")
train_identity=h2o.import_file("../input/train_identity.csv")
test_transaction=h2o.import_file("../input/test_transaction.csv")
test_identity=h2o.import_file("../input/test_identity.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
train= train_transaction.merge(train_identity, all_x=True)
test= test_transaction.merge(test_identity, all_x=True)

In [5]:
train.shape, train_transaction.shape,train_identity.shape, test.shape, test_transaction.shape, test_identity.shape

((590540, 434),
 (590540, 394),
 (144233, 41),
 (506691, 433),
 (506691, 393),
 (141907, 41))

# Let's treat the TransactionDT feature, per the dataset description given
References: https://www.kaggle.com/wajihullahbaig/up-sampling-on-every-hour

In [6]:
#Creates an hour of the day feature, encoded as 0-23. 
train.hours=((train['TransactionDT']/(3600)).floor())%24
test.hours=((test['TransactionDT']/(3600)).floor())%24

# Drop TransactionDT

In [7]:
X_train = train.drop(['TransactionDT'], axis=1)
X_test = test.drop(['TransactionDT'], axis=1)
del train_transaction, train_identity, test_transaction, test_identity
del train, test

# Find & tag categorical variables as H2O factor

In [8]:
for f in X_train.columns:
    if X_train[f].columns_by_type(coltype='categorical'): 
        X_train[f]=X_train[f].asfactor()

In [9]:
for f in X_test.columns:
    if X_test[f].columns_by_type(coltype='categorical'): 
        X_test[f]=X_test[f].asfactor()

In [10]:
X_train.pop('TransactionID')
X_test.pop('TransactionID')

TransactionID
3.66355e+06
3.66355e+06
3.66355e+06
3.66355e+06
3.66355e+06
3.66355e+06
3.66356e+06
3.66356e+06
3.66356e+06
3.66356e+06


In [11]:
x=X_train.columns[1:]
y='isFraud'
X_train[y]=X_train[y].asfactor()


# Lets try H2O autoML

In [12]:
%%time
aml = H2OAutoML(max_models=20, seed=18, max_runtime_secs=18000)
aml.train(x=x, y=y, training_frame=X_train)

AutoML progress: |████████████████████████████████████████████████████████| 100%
CPU times: user 1min 16s, sys: 6.37 s, total: 1min 22s
Wall time: 5h 9s


In [13]:
%%time
lb = aml.leaderboard
lb.head(rows=lb.nrows) 

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 7.36 ms


model_id,auc,logloss,mean_per_class_error,rmse,mse
XGBoost_1_AutoML_20190815_132750,0.949714,0.0645525,0.184026,0.123871,0.015344


In [14]:
%%time
aml.leader

CPU times: user 24 ms, sys: 8 ms, total: 32 ms
Wall time: 76.4 ms
Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_20190815_132750

Model Summary: 


,number_of_trees
,294.0




ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.012254022382807357
RMSE: 0.11069788788774318
LogLoss: 0.052426083866701105
Mean Per-Class Error: 0.08893829580432888
AUC: 0.9712954488085428
pr_auc: 0.8248278035302774
Gini: 0.9425908976170856
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.24299590332576862: 


,0,1,Error,Rate
0,567965.0,1912.0,0.0034,(1912.0/569877.0)
1,5863.0,14800.0,0.2837,(5863.0/20663.0)
Total,573828.0,16712.0,0.0132,(7775.0/590540.0)


Maximum Metrics: Maximum metrics at their respective thresholds



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
,2019-08-15 17:18:21,3:50:30.828,0.0,0.5,0.6931472,0.5,0.0,1.0,0.9650100
,2019-08-15 17:19:10,3:51:19.484,5.0,0.3990506,0.5089078,0.8584484,0.5365625,27.2923923,0.0258780
,2019-08-15 17:19:39,3:51:48.598,10.0,0.3237457,0.3891882,0.8681216,0.5497315,27.6069323,0.0249890
,2019-08-15 17:20:08,3:52:17.842,15.0,0.2679009,0.3065990,0.8735695,0.5714146,27.8007591,0.0241813
,2019-08-15 17:20:38,3:52:48.055,20.0,0.2271950,0.2477767,0.8779656,0.5873875,27.9456677,0.0236919
---,---,---,---,---,---,---,---,---,---
,2019-08-15 17:49:35,4:21:45.182,275.0,0.1116261,0.0532426,0.9703236,0.8224473,28.5650695,0.0134944
,2019-08-15 17:50:13,4:22:23.216,280.0,0.1113491,0.0529982,0.9705847,0.8243650,28.5650695,0.0134233
,2019-08-15 17:50:51,4:23:01.119,285.0,0.1110139,0.0527072,0.9709737,0.8260558,28.5699085,0.0132895
,2019-08-15 17:51:29,4:23:38.897,290.0,0.1108478,0.0525688,0.9711148,0.8232781,28.5747476,0.0133979



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
V258,53418.6914062,1.0,0.0762291
C1,27081.8183594,0.5069727,0.0386461
C14,25638.4375000,0.4799526,0.0365863
C13,24125.9375000,0.4516385,0.0344280
TransactionAmt,21076.9648438,0.3945616,0.0300771
---,---,---,---
V241,1.5487251,0.0000290,0.0000022
V110,1.1257648,0.0000211,0.0000016
id_12.missing(NA),0.8702278,0.0000163,0.0000012
M1.T,0.2701855,0.0000051,0.0000004



See the whole table with table.as_data_frame()


In [15]:
%%time
y_preds = aml.predict(X_test)
y_preds['p1'].as_data_frame().values.flatten().shape

xgboost prediction progress: |████████████████████████████████████████████| 100%


/opt/conda/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'DeviceInfo' has levels not trained on: [2014819, 2PS64, 4009A, 4027A, 4047A, 4047G, 5010G, 5011A, 5012G, 5015A, 5025G, 5026A, 5026A Build/NRD90M, 5033A, 5041C, 5044T, 5049W, 5049Z, 5052Y, 5054S, 5056A, 5058A, 5059A, 5080A, 5080X, 5085A, 5085B, 5086A, 5099A, 5099A Build/O00623, 6016A, 6036A Build/JLS36C, 6043A, 6045O, 6062W, 7040N, 7048A, 7048X, 8050G, 8080, 8082, 8_Plus, 9003A, 9024W, A10, A10 Pro Build/LMY47I, A1_PRO, A37fw, A462C, A502DL, A50LT, A622GL, A7, AGS-L03, AGS-L09, AGS-W09, ALCATELONETOUCH, ALE-L02, ALE-L21, ALE-L23, ALP-L09, ALP-L29, ANE-LX1, ANE-LX1 Build/HUAWEIANE-LX1, ANE-LX2, ANE-LX3, ASUS_A002A, ASUS_A006, ASUS_A007, ASUS_A009, ASUS_X008DC, ASUS_X00AD, ASUS_X00HD, ASUS_X00LDA, ASUS_X00PD, ASUS_Z00XS, ASUS_Z012DC, ASUS_Z017DC, ASUS_Z01HD, ASUS_Z01MD, ATU-L11, ATU-L21, ATU-LX3, ATU-LX3 Build/HUAWEIATU-LX3, AUM-L29, AW790, AX681, AX921, Alcatel_5044R, Alcatel_5098O, Alcatel

CPU times: user 528 ms, sys: 140 ms, total: 668 ms
Wall time: 18.4 s


(506691,)

In [16]:
%%time
import pandas as pd
sample_submission = pd.read_csv('../input/sample_submission.csv')

CPU times: user 108 ms, sys: 4 ms, total: 112 ms
Wall time: 116 ms


In [17]:
%%time
sample_submission['isFraud'] = y_preds['p1'].as_data_frame().values
sample_submission.to_csv('H2O_AutoML_submission_1.csv', index=False)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

CPU times: user 3.92 s, sys: 296 ms, total: 4.22 s
Wall time: 4.65 s
